In [57]:
import os
import requests
import alpaca_trade_api as tradeapi
import pandas as pd
from dotenv import load_dotenv


In [58]:
load_dotenv()

True

In [59]:
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')
print(type(alpaca_api_key))

<class 'str'>


In [60]:
alpaca=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2'
)

In [61]:
sectors_top5_dict = {
    'energy' : ['XOM', 'CVX', 'SHEL', 'TTE', 'COP'],
    'materials' : ['BHP', 'LIN', 'RIO', 'VALE', 'APD'],
    'industrials' : ['UPS', 'RTX', 'HON', 'CAT', 'UNP'],
    'utilities' : ['NEE', 'DUK', 'SO', 'D', 'SRE'],
    'healthcare' : ['UNH', 'JNJ', 'LLY', 'NVO', 'MRK'],
    'financials' : ['BRK-A', 'V', 'JPM', 'MA', 'BAC'],
    'consumer_discretionary' : ['AMZN', 'TSLA', 'HD', 'BABA', 'TM'],
    'consumer_staples' : ['WMT', 'PG', 'KO', 'PEP', 'COST'],
    'tech' : ['AAPL', 'MSFT', 'TSM', 'NVDA', 'ASML'],
    'communications' : ['GOOG', 'META', 'DIS', 'TMUS', 'CMCSA'],
    'real_estate' : ['PLD', 'AMT', 'EQIX', 'CCI', 'PSA'],
    'saas' : ['DDOG', 'NET', 'ZM', 'CRWD', 'ZS', 'ZI', 'BILL', 'MNDY', 'SNOW']
}

In [62]:
start_date = pd.Timestamp("2022-06-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-06-05", tz="America/New_York").isoformat()
timeframe='1Day'

In [63]:
# Create an empty dictionary to hold the dataframes of data for each sector.
sectors_df_dict = {} 

# Create a separate dataframe for each sector by looping through the keys of sectors_top5_dict.
for sector in sectors_top5_dict:
    sectors_df_dict[sector] = alpaca.get_bars(
        sectors_top5_dict[sector],
        timeframe,
        start = start_date,
        end = end_date
    ).df

# Create an empty dictionary to hold the dataframes of data for each ticker in the sector.
    ticker_dict = {}

# Enter each ticker's data into the ticker_dict dictionary
# where the key is the ticker and the value is the dataframe for that ticker.
    for ticker in sectors_top5_dict[sector]:
        ticker_dict[ticker] = sectors_df_dict[sector][sectors_df_dict[sector]['symbol']==ticker].drop(columns='symbol', axis=1)

# Concatenate the data for each ticker horizontally so each dataframe
# has a second level key with the name of the ticker. 
    ticker_df = pd.concat(ticker_dict.values(), axis=1, keys = sectors_top5_dict[sector])

# Add the resulting dataframe into the sectors_df_dict dictionary with the
# name of the sector as the key.
    sectors_df_dict[sector] = ticker_df

# Concatenate the dataframe for each sector so each has a third level key
# with the name of the sector.
sectors_df = pd.concat(sectors_df_dict.values(), axis = 1, keys = sectors_top5_dict.keys())
display(sectors_df)



energy                                    \
                             XOM                                     
                            open    high      low  close    volume   
timestamp                                                            
2021-06-01 04:00:00+00:00  59.45  60.695  59.4500  60.46  28469766   
2021-06-02 04:00:00+00:00  60.75  61.310  60.1700  60.94  22978321   
2021-06-03 04:00:00+00:00  60.55  61.410  60.3538  61.18  20267133   
2021-06-04 04:00:00+00:00  61.46  61.700  60.7758  61.45  20427479   

                                                                          ...  \
                                                     CVX                  ...   
                          trade_count       vwap    open    high     low  ...   
timestamp                                                                 ...   
2021-06-01 04:00:00+00:00      172953  60.259040  105.30  107.43  105.30  ...   
2021-06-02 04:00:00+00:00      146659  60.903259  107.21  108.56  106.21  ...   
2021-06-03 04:00:00+00:00      125159  61.057354  107.56  108.09  106.56  ...   
2021-06-04 04:00:00+00:00      122101  61.311344  108.24  108.67  107.21  ...   

                            saas                                           \
                            MNDY                     SNOW                   
                          volume trade_count vwap    open    high     low   
timestamp                                                                   
2021-06-01 04:00:00+00:00    NaN         NaN  NaN  239.50  243.56  235.51   
2021-06-02 04:00:00+00:00    NaN         NaN  NaN  240.74  248.20  239.60   
2021-06-03 04:00:00+00:00    NaN         NaN  NaN  240.00  243.23  233.36   
2021-06-04 04:00:00+00:00    NaN         NaN  NaN  240.73  244.86  237.50   

                                                                    
                                                                    
                            close   volume trade_count        vwap  
timestamp                                                           
2021-06-01 04:00:00+00:00  240.81  3772977       62221  239.968951  
2021-06-02 04:00:00+00:00  243.18  4016286       67209  243.649192  
2021-06-03 04:00:00+00:00  238.43  3598135       55131  238.985374  
2021-06-04 04:00:00+00:00  242.56  2630028       50951  242.648538  

[4 rows x 448 columns]

In [64]:
closing_prices_df = sectors_df.loc[:, (slice(None), slice(None), 'close')]
display(closing_prices_df)

energy                             materials  \
                             XOM     CVX  SHEL    TTE    COP       BHP   
                           close   close close  close  close     close   
timestamp                                                                
2021-06-01 04:00:00+00:00  60.46  106.65   NaN  47.76  58.27     76.30   
2021-06-02 04:00:00+00:00  60.94  108.08   NaN  48.62  58.99     76.25   
2021-06-03 04:00:00+00:00  61.18  107.59   NaN  48.60  59.47     74.95   
2021-06-04 04:00:00+00:00  61.45  108.45   NaN  48.57  59.85     76.12   

                                                         ... real_estate  \
                              LIN    RIO   VALE     APD  ...         PSA   
                            close  close  close   close  ...       close   
timestamp                                                ...               
2021-06-01 04:00:00+00:00  302.26  90.80  22.20  302.54  ...      285.56   
2021-06-02 04:00:00+00:00  299.88  90.22  22.67  305.66  ...      287.99   
2021-06-03 04:00:00+00:00  298.32  88.51  22.40  304.11  ...      288.85   
2021-06-04 04:00:00+00:00  299.86  89.75  22.45  304.76  ...      288.71   

                            saas                                        \
                            DDOG    NET      ZM    CRWD      ZS     ZI   
                           close  close   close   close   close  close   
timestamp                                                                
2021-06-01 04:00:00+00:00  91.88  83.07  327.72  222.07  191.77  42.76   
2021-06-02 04:00:00+00:00  91.60  83.86  327.11  220.58  191.34  43.49   
2021-06-03 04:00:00+00:00  88.69  81.92  319.01  216.00  184.25  41.82   
2021-06-04 04:00:00+00:00  88.45  83.08  336.00  206.94  185.80  44.20   

                                                 
                             BILL  MNDY    SNOW  
                            close close   close  
timestamp                                        
2021-06-01 04:00:00+00:00  151.59   NaN  240.81  
2021-06-02 04:00:00+00:00  155.94   NaN  243.18  
2021-06-03 04:00:00+00:00  146.39   NaN  238.43  
2021-06-04 04:00:00+00:00  148.48   NaN  242.56  

[4 rows x 64 columns]

In [65]:
daily_returns_df = closing_prices_df.pct_change()
display(daily_returns_df)

energy                                      \
                                XOM       CVX  SHEL       TTE       COP   
                              close     close close     close     close   
timestamp                                                                 
2021-06-01 04:00:00+00:00       NaN       NaN   NaN       NaN       NaN   
2021-06-02 04:00:00+00:00  0.007939  0.013408   NaN  0.018007  0.012356   
2021-06-03 04:00:00+00:00  0.003938 -0.004534   NaN -0.000411  0.008137   
2021-06-04 04:00:00+00:00  0.004413  0.007993   NaN -0.000617  0.006390   

                          materials                                          \
                                BHP       LIN       RIO      VALE       APD   
                              close     close     close     close     close   
timestamp                                                                     
2021-06-01 04:00:00+00:00       NaN       NaN       NaN       NaN       NaN   
2021-06-02 04:00:00+00:00 -0.000655 -0.007874 -0.006388  0.021171  0.010313   
2021-06-03 04:00:00+00:00 -0.017049 -0.005202 -0.018954 -0.011910 -0.005071   
2021-06-04 04:00:00+00:00  0.015610  0.005162  0.014010  0.002232  0.002137   

                           ... real_estate      saas                      \
                           ...         PSA      DDOG       NET        ZM   
                           ...       close     close     close     close   
timestamp                  ...                                             
2021-06-01 04:00:00+00:00  ...         NaN       NaN       NaN       NaN   
2021-06-02 04:00:00+00:00  ...    0.008510 -0.003047  0.009510 -0.001861   
2021-06-03 04:00:00+00:00  ...    0.002986 -0.031769 -0.023134 -0.024762   
2021-06-04 04:00:00+00:00  ...   -0.000485 -0.002706  0.014160  0.053259   

                                                                         \
                               CRWD        ZS        ZI      BILL  MNDY   
                              close     close     close     close close   
timestamp                                                                 
2021-06-01 04:00:00+00:00       NaN       NaN       NaN       NaN   NaN   
2021-06-02 04:00:00+00:00 -0.006710 -0.002242  0.017072  0.028696   NaN   
2021-06-03 04:00:00+00:00 -0.020763 -0.037054 -0.038400 -0.061242   NaN   
2021-06-04 04:00:00+00:00 -0.041944  0.008412  0.056911  0.014277   NaN   

                                     
                               SNOW  
                              close  
timestamp                            
2021-06-01 04:00:00+00:00       NaN  
2021-06-02 04:00:00+00:00  0.009842  
2021-06-03 04:00:00+00:00 -0.019533  
2021-06-04 04:00:00+00:00  0.017322  

[4 rows x 64 columns]